<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №1</center>

<center>«Введение в MapReduce модель на Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Фамилия И.О.
<br>613х-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>


# Введение в MapReduce модель на Python


# **Модель MapReduce**
Функция MapReduce имитирует работу фреймворка MapReduce: подготовка и форматирование исходных данных в виде набора пар ключ-значение (key-value) для последующего вызова функции MAP, вызов функции MAP, группировка промежуточных результатов работы функции MAP по ключу и формирование массива значений для каждого ключа, вызов функции REDUCE.

Пользователь для решения своей задачи реализует функции RECORDREADER, MAP, REDUCE.

In [3]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




## WordCount (пример)

In [7]:
from typing import Iterator

d1 = """it is what it is
it is what it is
it is what it is"""
d2 = """what is it
what is it"""
d3 = """it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

## Вычисление TF-IDF (Term Frequency – Inverse Document Fraquency)

Реализуется в три этапа:

**Этап 1:** Частота слова в документе

**Этап 2:** Количество документов, в которых встречается слово

**Этап 3:** Расчёт TF-IDF

Реализация включает:
1. Формирование набора исходных данных: проиндексированные строки, содержащие текст аннотаций.
2. Код функций RECORDREADER_1, MAP_1, REDUCE_1 для этапа 1.
3. Код функций RECORDREADER_2, MAP_2, REDUCE_2 для этапа 2.
4. Код функций RECORDREADER_3, MAP_3, REDUCE_3 для этапа 3.
5. Последовательный вызов модельной функции Mapreduce для всех трёх этапов.
6. Ввод на печать результата отдельно для каждого документа в виде первых пяти слов и их TF-IDF, упорядоченных по убыванию их значений (для каждого документа вывести на печать первые пять слов с максимальным TF-IDF).

In [ ]:
# Код для чтения данных из DOCX файла и реализации TF-IDF
import zipfile
import xml.etree.ElementTree as ET
import re
import math
from collections import defaultdict
from typing import Iterator
from pathlib import Path

def _docx_paragraphs(path):
    # Открываем .docx как zip-архив и читаем XML основного документа
    with zipfile.ZipFile(path) as z:
        xml = z.read("word/document.xml")
    # Пространство имён WordprocessingML
    ns = {"w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}
    # Парсим XML и собираем список параграфов
    root = ET.fromstring(xml)
    paras = []
    for p in root.findall(".//w:p", ns):
        # Внутри абзаца собираем все текстовые узлы <w:t>
        texts = [t.text for t in p.findall(".//w:t", ns) if t.text]
        # Склеиваем кусочки в одну строку абзаца (или пустую строку)
        paras.append("".join(texts) if texts else "")
    # Нормализуем пробелы в каждом параграфе: сжимаем подряд и обрезаем края
    return [re.sub(r"\s+", " ", t).strip() for t in paras]

def parse_lab_data_from_docx(path):
    # Получаем плоский список абзацев (уже без лишних пробелов)
    paras = _docx_paragraphs(path)
    
    # -------- Извлечение документов (аннотаций) --------
    docs, current = [], []
    in_docs = False
    
    for line in paras:
        # Заголовок раздела аннотации (начало нового документа)
        if re.search(r"^\s*Аннотац", line, re.IGNORECASE):
            # если накапливался предыдущий документ - сохраняем
            if current:
                docs.append(" ".join(current).strip())
                current = []
            in_docs = True
            continue
            
        if in_docs:
            # дошли до раздела со списком смежности - значит, аннотация закончилась
            if re.search(r"Список\s+смежности", line, re.IGNORECASE):
                break
            # копим непустые строки текущего документа
            if line.strip():
                current.append(line.strip())
    
        # финальный "хвост" документа, если остался
    if current:
        docs.append(" ".join(current).strip())
    
    # -------- Извлечение списка смежности --------
    adj = {}
    in_adj = False
    
    for line in paras:
        # Начало раздела "Список смежности"
        if re.search(r"Список\s+смежности", line, re.IGNORECASE):
            in_adj = True
            continue
        
        if in_adj and line.strip():
            # Парсим строку вида "0: [1, 3]"
            match = re.match(r"(\d+):\s*\[(.*?)\]", line.strip())
            if match:
                node_id = match.group(1)
                neighbors_str = match.group(2)
                # Извлекаем номера соседних узлов
                neighbors = []
                if neighbors_str.strip():
                    neighbor_nums = re.findall(r"\d+", neighbors_str)
                    neighbors = [int(n) for n in neighbor_nums]
                adj[node_id] = neighbors
    
    return docs, adj

# Читаем данные из DOCX файла
try:
    docx_path = Path("tasks/Lab1_данные.docx")
    documents, adj = parse_lab_data_from_docx(docx_path)
    print(f"Успешно прочитано {len(documents)} документов из {docx_path}")
    for i, doc in enumerate(documents):
        print(f"Документ {i}: {doc[:100]}...")
except Exception as e:
    print(f"Ошибка чтения DOCX файла: {e}")
    # Fallback данные для тестирования
    documents = [
        "The quick brown fox jumps over the lazy dog",
        "A quick brown dog jumps over a lazy fox",
        "The lazy brown dog is quick and jumps over the fox"
    ]
    print("Используем тестовые данные")

# Настройки для TF-IDF
N_DOCS = len(documents)
print(f"Общее количество документов: {N_DOCS}")

# Токенайзер: слова на латинице, допускаем дефис и апостроф внутри слова
TOKEN_RE = re.compile(r"[A-Za-z][A-Za-z\-']+")

def tokenize(text: str):
    # Находим все токены по шаблону и приводим к нижнему регистру
    return [w.lower() for w in TOKEN_RE.findall(text)]

# ====== ЭТАП 1: Частота слова в документе ======

def RECORDREADER_1():
    for doc_id, text in enumerate(documents):
        yield (doc_id, text)

def MAP_1(key, value: str):
    doc_id = key
    toks = tokenize(value)
    for tok in toks:
        yield ((doc_id, tok), 1)
    # Подсчёт total токенов на документ
    yield ((doc_id, "*CNT*"), len(toks))

def REDUCE_1(key, values: Iterator[int]):
    s = 0
    for v in values:
        s += v
    yield (key, s)

print("Выполняем этап 1 TF-IDF...")
stage1_out = list(MapReduce(RECORDREADER_1, MAP_1, REDUCE_1))
print(f"Этап 1 завершен. Получено {len(stage1_out)} записей")

# ====== ЭТАП 2: Количество документов, в которых встречается слово ======

def RECORDREADER_2():
    for kv in stage1_out:
        yield kv

def MAP_2(key, value):
    # key == (doc_id, tok) OR (doc_id, "*CNT*")
    doc_id, tok = key
    if tok == "*CNT*":
        yield (("TOTAL", doc_id), value)
    else:
        # отмечаем факт присутствия терма в документе
        yield ((tok, doc_id), 1)

def REDUCE_2(key, values: Iterator[int]):
    if key[0] == "TOTAL":
        total = 0
        for v in values:
            total += v
        yield (key, total)
    else:
        tok, doc_id = key
        cnt = 0
        for v in values:
            cnt += v
        if cnt > 0:
            yield (("DF", tok), doc_id)

print("Выполняем этап 2 TF-IDF...")
stage2_out = list(MapReduce(RECORDREADER_2, MAP_2, REDUCE_2))
print(f"Этап 2 завершен. Получено {len(stage2_out)} записей")

# ====== ЭТАП 3: Расчёт TF-IDF ======

def RECORDREADER_3():
    for kv in stage1_out:
        yield kv
    for kv in stage2_out:
        yield kv

def MAP_3(key, value):
    # stage1: ((doc, tok), cnt) | ((doc, "*CNT*"), total)
    # stage2: (("TOTAL", doc), total) | (("DF", tok), doc_id)
    if isinstance(key, tuple) and len(key) == 2 and isinstance(key[0], int):
        doc_id, tok = key
        if tok == "*CNT*":
            yield (("TOTAL", doc_id), value)
        else:
            yield (("TF", doc_id, tok), ("CNT", value))
    elif key[0] == "TOTAL":
        _, doc_id = key
        yield (("TOTAL", doc_id), value)
    elif key[0] == "DF":
        _, tok = key
        yield (("DFTOK", tok), value)

def REDUCE_3(group_key, values: Iterator):
    if group_key[0] == "TOTAL":
        total = 0
        for v in values:
            total += v
        yield (group_key, total)
    elif group_key[0] == "DFTOK":
        seen = set(values)
        df = len(seen)
        yield (("DF", group_key[1]), df)
    elif group_key[0] == "TF":
        cnt = 0
        for tag, v in values:
            if tag == "CNT":
                cnt += v
        yield (group_key, cnt)

print("Выполняем этап 3 TF-IDF...")
stage3_intermediate = list(MapReduce(RECORDREADER_3, MAP_3, REDUCE_3))

# Собираем TOTAL, DF, TF и считаем tf-idf
totals, dfs, tfs_raw = {}, {}, {}
for key, val in stage3_intermediate:
    if key[0] == "TOTAL":
        totals[key[1]] = val
    elif key[0] == "DF":
        dfs[key[1]] = val
    elif key[0] == "TF":
        _, doc_id, tok = key
        tfs_raw[(doc_id, tok)] = val

tfidf = defaultdict(dict)
for (doc_id, tok), tn in tfs_raw.items():
    cn = totals.get(doc_id, 1)
    tf = tn / cn if cn else 0.0
    df = dfs.get(tok, 1)
    idf = math.log(N_DOCS / df) if df > 0 else 0.0
    tfidf[doc_id][tok] = tf * idf

# Выводим топ-5 слов для каждого документа
print("\nРезультаты TF-IDF - топ-5 слов для каждого документа:")
top5_per_doc = {doc_id: sorted(scores.items(), key=lambda kv: kv[1], reverse=True)[:5]
                for doc_id, scores in tfidf.items()}

for doc_id, pairs in sorted(top5_per_doc.items()):
    print(f"Документ {doc_id}: {[(w, round(s,6)) for w,s in pairs]}")

## Поиск кратчайшего пути на графе с использованием алгоритма BFS

Реализация включает:
1. Формирование набора исходных данных: структура, содержащая списки смежности вершин исходного графа.
2. Указание начальной и искомой вершин пути на графе.
3. Код функции RECORDREADER, формирующей набор исходных данных для каждой итерации, в том числе и для первой итерации.
4. Код функций  MAP и REDUCE для каждой итерации.
5. Код функции, формирующей признак завершения итераций.
6. Код цикла итераций алгоритма BFS, с вызовом модельной функции Mapreduce в теле цикла.
7. Вывод на печать кратчайшего пути на графе от исходной до искомой вершины.